In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm, trange
import firedrake
from firedrake import (
    max_value, min_value, Constant, inner, grad, div, sym, tr, dx, ds
)
import irksome
from irksome import Dt

In [ ]:
lx = 1.0
nx = 32
interval = firedrake.IntervalMesh(nx, lx)

In [ ]:
hermite = firedrake.FiniteElement("Hermite", "interval", 3)
Q = firedrake.FunctionSpace(interval, hermite)

In [ ]:
cg3 = firedrake.FiniteElement("CG", "interval", 3)
P = firedrake.FunctionSpace(interval, cg3)

In [ ]:
x, = firedrake.SpatialCoordinate(interval)

h = firedrake.Function(Q)
h_0 = Constant(1.0)
h.project(h_0)

t = Constant(0.0)
Lx = Constant(lx)
T = Constant(1.0)
u_expr = firedrake.sin(2 * π * x / Lx) * firedrake.sin(2 * π * t / T)
u = firedrake.as_vector((u_expr,))

In [ ]:
ϕ = firedrake.TestFunction(Q)
F_cells = (Dt(h) * ϕ - inner(h * u, grad(ϕ))) * dx
n = firedrake.FacetNormal(interval)
F_inflow = h_0 * min_value(0, inner(u, n)) * ϕ * ds
F_outflow = h * max_value(0, inner(u, n)) * ϕ * ds
F = F_cells + F_inflow + F_outflow

In [ ]:
timestep = 0.01
dt = Constant(timestep)

method = irksome.RadauIIA(2)
solver = irksome.TimeStepper(F, method, t, dt, h)

In [ ]:
final_time = 4 * float(T)
num_steps = int(final_time / timestep)

hs = [firedrake.Function(P).project(h)]
for step in trange(num_steps):
    solver.advance()
    t.assign(t + dt)
    hs.append(firedrake.Function(P).project(h))

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim((0, 2))
firedrake.plot(hs[0], axes=ax)

def animate(h):
    ax.clear()
    firedrake.plot(h, axes=ax)
    ax.set_ylim((0, 2))

animation = FuncAnimation(fig, animate, tqdm(hs), interval=1e3/24)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

In [ ]:
volumes = np.array([firedrake.assemble(h * dx) for h in hs])
volumes.min(), volumes.max()